In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import pickle

In [72]:
# create list of mETRO and not MICRO politan SAs
reference_table = pd.read_csv("./cbsa-state-mapping.csv", sep=";")
reference_table['CBSA Code'] = reference_table['CBSA Code'].astype(int) / 10
reference_table['CBSA Code'] = ('C' + reference_table['CBSA Code'].astype(int).astype(str))
relevant_MSAs = reference_table[reference_table['Metropolitan/Micropolitan Statistical Area'].
                                str.contains("Metropolitan")]['CBSA Code']
# don't consider micropolitan areas
relevant_MSAs = relevant_MSAs.drop_duplicates().tolist()

In [4]:
treated_msas = ['C4194', 'C4266', 'C4162', 'C4174', 'C1450', 'C3070', 'C2642', 'C1698', 'C1658', 'C1146', 'C3830', 'C2706', 'C1206', 'C4790', 'C1446', 'C4594']

outcomes_to_test = ['lq_annual_avg_estabs', 'lq_annual_avg_emplvl',
       'lq_avg_annual_pay', 'annual_avg_estabs', 'annual_avg_emplvl', 'avg_annual_pay']

industries_to_test = ['51', '511', '512', '515', '516', '517', '518', '519']

In [5]:
# SET UP CBSA REFERENCE TABLE
reference_table = pd.read_csv("./cbsa-state-mapping.csv", sep=";")
reference_table['CBSA Code'] = reference_table['CBSA Code'].astype(int) / 10
reference_table['CBSA Code'] = ('C' + reference_table['CBSA Code'].astype(int).astype(str))
reference_table['area_fips'] = reference_table['CBSA Code']
reference_table['state'] = reference_table['State Name']
reference_table = reference_table[['area_fips', 'State Name']].drop_duplicates()

In [7]:
# COMPILE RELEVANT INDUSTRY OUTCOMES AND PUT INTO A PANEL DATA FORMAT

for industry in industries_to_test:
    dfs = []
    print(industry)
    for year in range(1990, 2019):
        df = pd.read_csv('./original_data/' + str(year) + '.annual.singlefile.csv')
        df = df[['area_fips', 'industry_code', 'lq_annual_avg_estabs', 'lq_annual_avg_emplvl',
           'lq_avg_annual_pay', 'annual_avg_estabs', 'annual_avg_emplvl', 'avg_annual_pay']]
        df = df[df['area_fips'].astype(str).str.contains(pat="([C][0-9][0-9][0-9][0-9])", regex=True, na=False)]
        df = df[df['industry_code'].astype(str).str.contains(pat="^" + industry + "$", regex=True, na=False)]
        df['year'] = year
        dfs.append(df)
    master_df = pd.concat(dfs, axis=0)

    # now, need to groupby the year+area_fips and sort by year
    group = master_df.sort_values(['year'],ascending=True).groupby(['area_fips', 'year'])
    master_df = group.agg({'lq_annual_avg_estabs':sum, 'lq_annual_avg_emplvl':sum, 'lq_avg_annual_pay':sum,
                           'annual_avg_estabs':sum, 'annual_avg_emplvl':sum, 'avg_annual_pay':sum}).reset_index()
    
    # add on the state that the MSA is in
    merged = pd.merge(master_df, reference_table, how='left', on="area_fips")
    merged.to_csv("panel_" + industry + "_msa.csv")

51


/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,7,15,23) have mixed types. Specify dtype option on import or set low_memo

511


/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=Fa

512


...

/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=Fa

518


/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=Fa

519


/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=Fa

In [8]:
county_ref_table = pd.read_csv("./R_Files/county_latlongs.csv")

In [11]:
county_ref_table['area_fips'] = county_ref_table['fips'] 
county_ref_table = county_ref_table[['area_fips', 'state']]

In [12]:
# REPEAT THE SAME AT MORE GRANULAR COUNTY LEVEL - NOT FINISHED, NEED COUNTY REFERENCE TABLE FOR STATE FE

# COMPILE RELEVANT INDUSTRY OUTCOMES AND PUT INTO A PANEL DATA FORMAT

for industry in industries_to_test:
    dfs = []
    print(industry)
    for year in range(1990, 2019):
        df = pd.read_csv('./original_data/' + str(year) + '.annual.singlefile.csv')
        df = df[['area_fips', 'industry_code', 'lq_annual_avg_estabs', 'lq_annual_avg_emplvl',
           'lq_avg_annual_pay', 'annual_avg_estabs', 'annual_avg_emplvl', 'avg_annual_pay']]
        df = df[df['area_fips'].astype(str).str.isdigit()]
        df = df[df['industry_code'].astype(str).str.contains(pat="^" + industry + "$", regex=True, na=False)]
        df['year'] = year
        dfs.append(df)
    master_df = pd.concat(dfs, axis=0)

    # now, need to groupby the year+area_fips and sort by year
    group = master_df.sort_values(['year'],ascending=True).groupby(['area_fips', 'year'])
    master_df = group.agg({'lq_annual_avg_estabs':sum, 'lq_annual_avg_emplvl':sum, 'lq_avg_annual_pay':sum,
                           'annual_avg_estabs':sum, 'annual_avg_emplvl':sum, 'avg_annual_pay':sum}).reset_index()
    
    # add on the state that the MSA is in
    merged = pd.merge(master_df, county_ref_table, how='left', on="area_fips")
    merged.to_csv("panel_" + industry + "_county.csv")

51


/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,7,15,23) have mixed types. Specify 

511


/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,7,15,23) have mixed types. Specify 

512


...

/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,7,15,23) have mixed types. Specify 

518


/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,7,15,23) have mixed types. Specify 

519


/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gaoag/.virtualenvs/machinelearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,7,15,23) have mixed types. Specify 

In [68]:
outcomes = ['lq_annual_avg_estabs', 'lq_annual_avg_emplvl', 'lq_avg_annual_pay', 'annual_avg_estabs', 'annual_avg_emplvl', 'avg_annual_pay']

code_state_pairs = [(('C4194', 'Palo Alto'), 'California'), (('C4174', 'San Diego'), 'California'),
   (('C4162', 'Salt Lake City'), 'Utah'), (('C1450', 'Boulder'), 'Colorado'), (('C4266', 'Seattle'), 'Washington'),
   (('C3070', 'Lincoln'), 'Nebraska'), (('C2642', 'Houston'), 'Texas'), (('C1146', 'Ann Arbor'), 'Michigan'),
   (('C3830', 'Pittsburgh'), 'Pennsylvania'), (('C2706', 'Ithaca'), 'New York'), (('C1206', 'Atlanta'), 'Georgia'),
   (('C1446', 'Boston'), 'Massachusetts'), (('C4594', 'Princeton'), 'New Jersey')]

for ind in ['511', '512', '515', '516', '517', '518']:
    df = pd.read_csv("./msa_files/panel_"+ ind + "_msa.csv")
    for outcome in outcomes:  
        for msa, state in code_state_pairs:
            msa_outcome_to_graph = df[df['area_fips'] == msa[0]].set_index('year')[outcome]
            avg_outcome_to_graph = df[df['State Name'] == state].groupby('year').agg({outcome: "mean"})[outcome]
            msa_outcome_to_graph = msa_outcome_to_graph[msa_outcome_to_graph != 0]
            plt.figure()
            plt.plot(msa_outcome_to_graph, color="orange")
            plt.plot(avg_outcome_to_graph, color="blue")
            plt.title(msa[1] + " against average " + state + " " + outcome + " - sector " + ind)
            plt.savefig('./graphs/' + ind + "/" + outcome + "/" + msa[1] + ".png")
            plt.close('all')

In [70]:
outcomes = ['lq_annual_avg_estabs', 'lq_annual_avg_emplvl', 'lq_avg_annual_pay', 'annual_avg_estabs', 'annual_avg_emplvl', 'avg_annual_pay']

code_state_pairs = [(('C4194', 'Palo Alto'), 'California'), (('C4174', 'San Diego'), 'California'),
   (('C4162', 'Salt Lake City'), 'Utah'), (('C1450', 'Boulder'), 'Colorado'), (('C4266', 'Seattle'), 'Washington'),
   (('C3070', 'Lincoln'), 'Nebraska'), (('C2642', 'Houston'), 'Texas'), (('C1146', 'Ann Arbor'), 'Michigan'),
   (('C3830', 'Pittsburgh'), 'Pennsylvania'), (('C2706', 'Ithaca'), 'New York'), (('C1206', 'Atlanta'), 'Georgia'),
   (('C1446', 'Boston'), 'Massachusetts'), (('C4594', 'Princeton'), 'New Jersey')]

for ind in ['511', '512', '515', '516', '517', '518']:
    df = pd.read_csv("./msa_files/panel_"+ ind + "_msa.csv")
    for outcome in outcomes:  
        for msa, state in code_state_pairs:
            msa_outcome_to_graph = df[df['area_fips'] == msa[0]].set_index('year')[outcome]
            avg_outcome_to_graph = df[df['State Name'] == state].groupby('year').agg({outcome: "mean"})[outcome]
            msa_outcome_to_graph = msa_outcome_to_graph[msa_outcome_to_graph != 0]
            plt.figure()
            plt.plot(msa_outcome_to_graph, color="orange")
            plt.plot(avg_outcome_to_graph, color="blue")
            plt.title(msa[1] + " against average " + state + " " + outcome + " - sector " + ind)
            plt.savefig('./graphs/' + ind + "/" + outcome + "/" + msa[1] + ".png")
            plt.close('all')